In [2]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [4]:
# 데이터 로드
article_df = pd.read_csv('article.csv')
article_df.tail(2)

,category,content,link,title
598,105,병역 대체복무제도 개선 확정…중소·중견기업에 배정 늘려(세종=연합뉴스) 신선미 기자...,https://news.naver.com/main/read.nhn?mode=LSD&...,2025년 전문연구요원 석사 300명 감축…박사는 1천명 유지(종합)
599,105,엔지니어들이 SOC실증연구센터에서 스마트 건설 방식으로 공사를 진행하고 있다. 제...,https://news.naver.com/main/read.nhn?mode=LSD&...,"SKT, ‘5G+건설·기계’ 드론 띄워 측량, 굴삭기로 정밀공사 ‘착착’"


In [8]:
# 데이터 전처리 : 데이터 셋 분리
x_train, x_test, y_train, y_test = train_test_split(
article_df.content, article_df.category, test_size=0.1, random_state=1)
len(x_train), len(x_test), len(y_train), len(y_test)

(540, 60, 540, 60)

In [9]:
# 모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1))
])

In [11]:
# 학습
model = clf.fit(x_train.values.astype('str'), y_train)

In [13]:
# 모델 성능 확인
y_pred = model.predict(x_test)

In [14]:
list(y_test[:5]), list(y_pred[:5])

([104, 104, 105, 104, 102], [104, 104, 105, 104, 102])

In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         100       0.60      0.90      0.72        10
         101       0.56      0.62      0.59         8
         102       0.43      0.30      0.35        10
         103       1.00      0.29      0.44         7
         104       1.00      0.81      0.90        16
         105       0.64      1.00      0.78         9

    accuracy                           0.68        60
   macro avg       0.70      0.65      0.63        60
weighted avg       0.73      0.68      0.67        60



In [16]:
# 모델 사용하기
categories = {
    100 : '정치',
    101 : '경제',
    102 : '사회', 
    103 : '생활/문화',
    104 : '세계',
    105 : 'IT/과학'
}

In [17]:
contents = [
    '네이버와 카카오는 드론 기술 발전에 주력',
    '요즘 환율 주가 예측이 불가',
    '광화문 대한애국당 천막, 격령 저항 속 철거'
]

In [18]:
datas = {
    'content' : contents,
    'category_code' : model.predict(contents)
}
df = pd.DataFrame(datas)
df

,content,category_code
0,네이버와 카카오는 드론 기술 발전에 주력,105
1,요즘 환율 주가 예측이 불가,100
2,"광화문 대한애국당 천막, 격령 저항 속 철거",103


In [23]:
round(max(model.predict_proba([contents[0]])[0]), 2)

0.5

In [25]:
df['category'] = df['category_code'].apply(lambda data: categories[data])
df['proba'] = df['content'].apply(
    lambda data: round(max(model.predict_proba([data])[0]), 2))
df

,content,category_code,category,proba
0,네이버와 카카오는 드론 기술 발전에 주력,105,IT/과학,0.50
1,요즘 환율 주가 예측이 불가,100,정치,0.22
2,"광화문 대한애국당 천막, 격령 저항 속 철거",103,생활/문화,0.17


In [26]:
# 모델 저장
pickle.dump(model, open('clf.pkl', 'wb'))